In [13]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from PIL import Image, ImageFile

In [ ]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
# validating images
def validate_images(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                img = Image.open(file_path)
                img.verify()
            except (IOError, SyntaxError) as e:
                print(f"Corrupted file removed: {file_path}")
                os.remove(file_path)

In [ ]:
# train and test folders
validate_images('DATASET/TRAIN')
validate_images('DATASET/TEST')

# paths for directories
train_dir = 'DATASET/TRAIN'
test_dir = 'DATASET/TEST'

In [ ]:
# img params
image_size = (224, 224)
batch_size = 32
epochs = 10
num_classes = len(os.listdir(train_dir))

In [ ]:
# Data Augumentation for cleaning data
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [19]:
test_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 1081 images belonging to 5 classes.


In [20]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 470 images belonging to 5 classes.


In [ ]:
# adding layers

# Loading a pre-trained MobileNetV2 model without the top layer
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freezing base model layers
base_model.trainable = False

# Adding custom layers on top
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=output)

# Compiling the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# training 
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=test_generator
)

C:\Users\ganes\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 78s 2s/step - accuracy: 0.3249 - loss: 2.2636 - val_accuracy: 0.8851 - val_loss: 0.5073
Epoch 2/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 86s 3s/step - accuracy: 0.5754 - loss: 1.0847 - val_accuracy: 0.9277 - val_loss: 0.3126
Epoch 3/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 86s 3s/step - accuracy: 0.6619 - loss: 0.8889 - val_accuracy: 0.9447 - val_loss: 0.2501
Epoch 4/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 90s 3s/step - accuracy: 0.7394 - loss: 0.7304 - val_accuracy: 0.9426 - val_loss: 0.1996
Epoch 5/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 88s 3s/step - accuracy: 0.7563 - loss: 0.6780 - val_accuracy: 0.9426 - val_loss: 0.1917
Epoch 6/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 87s 3s/step - accuracy: 0.7674 - loss: 0.6378 - val_accuracy: 0.9596 - val_loss: 0.1299
Epoch 7/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 90s 3s/step - accuracy: 0.7940 - loss: 0.5595 - val_accuracy: 0.9660 - val_loss: 0.1135
Epoch 8/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 99s 3s/step - accuracy: 0.7727 - loss: 0.6400 - val_accuracy: 0.9660 - val_loss:

In [ ]:
# Evaluation
loss, accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

15/15 ━━━━━━━━━━━━━━━━━━━━ 24s 2s/step - accuracy: 0.9628 - loss: 0.1276
Test Accuracy: 96.38%


In [ ]:
model.save('pose_detection_model.h5')